This Jupyter notebook showcases fundamental SQLAlchemy skills for managing a database. It includes:

1. **Database and Table Creation**: Demonstrates how to define and create tables in a SQLite database using SQLAlchemy.
2. **Data Insertion**: Illustrates the process of inserting records into tables.
3. **Data Update**: Shows how to update existing records based on specific conditions.
4. **Data Deletion**: Details how to delete records from tables according to given criteria.

Overall, the notebook provides a practical guide to creating, manipulating, and managing a database with SQLAlchemy

In [46]:
from sqlalchemy import create_engine, MetaData, Table, select, func, distinct, case, cast, Float
from sqlalchemy import delete, update, String, Integer
import pandas as pd

# Creating database

In [3]:
# create the engine
engine = create_engine("sqlite:///ecommerce.db")
connection = engine.connect()
metadata = MetaData()

In [4]:
# create an order table
order = Table("order", metadata, 
              Column("order_id", Integer(), unique=True, nullable=False), 
              Column ("user_id", Integer(), nullable= False), 
              Column("product_id", Integer()), 
              Column ("quantity", Integer()), 
              Column ("order_date", Date()),  extend_existing=True)

# create  a user table
user = Table("user", metadata, 
             Column ("user_id", Integer(), nullable=False), 
             Column ("username", String(), nullable= False), 
             Column("email", String(), nullable = False), 
             Column ("age", Integer()), 
             Column ("country", String()), extend_existing = True)

# create a product table 
product = Table("product", metadata, 
                Column("product_id", Integer(), nullable=False), 
                Column("product_name", String()), 
                Column("price", Float()),extend_existing = True)
# create a payment table
payment =  Table("payment", metadata, 
                Column("payment_id", Integer(), nullable=False, unique=True), 
                Column("order_id",Integer(), nullable= False),
                Column("payment_date", Date(), nullable = False),
                Column("amount", Float()),
                 Column("payment_method", String()),extend_existing = True)

metadata.create_all(engine)

# Inserting data

In [5]:
 # Inserting order data into a table 
stmt = insert(order)
values_list = [
   {"order_id": 1001, "user_id": 111, "product_id": 1, "quantity": 200, "order_date": date(2024, 9, 1)}, 
    {"order_id": 1002, "user_id": 112, "product_id": 2, "quantity": 203, "order_date": date(2024, 10, 1)}, 
    {"order_id": 1003, "user_id": 113, "product_id": 3, "quantity": 560, "order_date": date(2023, 9, 1)}, 
    {"order_id": 1004, "user_id": 114, "product_id": 4, "quantity": 2040, "order_date": date(2022, 9, 1)}
]
result_proxy = connection.execute(stmt, values_list)
connection.commit() 
print(result_proxy.rowcount)

4


In [6]:
 # Inserting product data into a table 
stmt = insert(product)
values_list = [
   {"product_id":1, "product_name": "Laptop", "price": 1569},
    {"product_id":2, "product_name": "Smartphone", "price": 1245},
    {"product_id":3, "product_name": "Tablet", "price": 456},
    {"product_id":4, "product_name": "headphones", "price": 169}]
result_proxy = connection.execute(stmt, values_list)
connection.commit() 
print(result_proxy.rowcount)

4


In [7]:
 # Inserting payment data into a table 
stmt = insert(payment)
values_list = [
   {"payment_id": 5001,	"order_id": 1001, "payment_date":date(2024, 9, 2), "amount": 1234,"payment_method": "Credit Card"},
{"payment_id": 5002,	"order_id": 1002, "payment_date":date(2024, 10, 2), "amount": 123,"payment_method": "Credit Card"},
{"payment_id": 5003,	"order_id": 1003, "payment_date":date(2023, 9, 2), "amount": 300,"payment_method": "PayPal"},
{"payment_id": 5004,	"order_id": 1004, "payment_date":date(2024, 9, 2), "amount": 45,"payment_method": "Bank Transfer"}]
result_proxy = connection.execute(stmt, values_list)
connection.commit() 
print(result_proxy.rowcount)

4


In [9]:
 # Inserting user data into a table 
stmt = insert(user)
values_list = [
   {"user_id": 111,	"username":"Maika" , "email":"Maika@.hotmail.com", "age": 19, "Country": "USA"},
{"user_id": 112,	"username": "Lar", "email":"lar@.hotmail.com", "age": 22, "Country": "Swede"},
{"user_id": 113,	"username": "Ahmed", "email":"Ahmed@.hotmail.co", "age": 40, "Country": "Netherlands"},
{"user_id": 114,	"username": "Amiir", "email":"Amiir@.hotmail.com", "age": 65, "Country": "USA"}]
result_proxy = connection.execute(stmt, values_list)
connection.commit() 
print(result_proxy.rowcount)

4


 # Updating data in a table

In [12]:
# load the tables
engine = create_engine("sqlite:///ecommerce.db")
connection = engine.connect()
order_table = Table("order", metadata, autoload_with = engine)
user_table = Table("user", metadata, autoload_with = engine)
product_table = Table("product", metadata, autoload_with = engine)
payment_table = Table("payment", metadata, autoload_with = engine)

In [14]:
# print the column names 
print(order_table.columns.keys())
print(product_table.columns.keys())
print(user_table.columns.keys())
print(payment_table.columns.keys())

['order_id', 'user_id', 'product_id', 'quantity', 'order_date']
['product_id', 'product_name', 'price']
['user_id', 'username', 'email', 'age', 'country']
['payment_id', 'order_id', 'payment_date', 'amount', 'payment_method']


Update the mail address of the user with `user_id`112 to `lar_new@hotmail.com`.

In [23]:
# Select statement 
select_stmt = select(user_table).where(user_table.columns.user_id ==112)

# Update statement
update_stmt = update(user_table).values(email = 'lar_new@hotmail.com')
update_stmt = update_stmt.where(user_table.columns.user_id == 112)

# Execute the results
update_results = connection.execute(update_stmt)

# Execute and print the results to see the changes
new_results = connection.execute(select_stmt).fetchall()
print(new_results)

[(112, 'Lar', 'lar_new@hotmail.com', 22, None)]


Increase the price of all products  by `10%`.

In [25]:
# Select statement 
select_stmt = select(product_table)

# Update statement 
update_stmt = update(product_table).values(price = product_table.columns.price * 1.11)

# Execute the results 
update_results = connection.execute(update_stmt)

# Execute and print the rsults to see the changes 
new_results = connection.execute(select_stmt).fetchall()
print(new_results)

[(1, 'Laptop', 1741.5900000000001), (2, 'Smartphone', 1381.95), (3, 'Tablet', 506.16), (4, 'headphones', 187.59)]


Change the quantity of the order with `order_id 1003 to 700`.

In [29]:
# Select statement 
select_stmt = select(order_table).where(order_table.columns.order_id == 1003)

# Update statement 
update_stmt = update(order_table).values(quantity = 700).where (order_table.columns.order_id ==1003)

# Execute the update statement 
update_stmt = connection.execute(update_stmt)

# Execute and print the new results
update_results = connection.execute(select_stmt).fetchall()
print(update_results)

[(1003, 113, 3, 700, datetime.date(2023, 9, 1))]


Apply a `15% discount` to the price of all products where the price is greater than $1000.

In [31]:
# Select statement 
select_stmt = select(product_table)

# Update statement 
update_stmt = update(product_table).values(price = product_table.columns.price * 0.85
                                          ).where(product_table.columns.price > 1000)

# Execute the update statemnt 
update_stmt = connection.execute(update_stmt)

# Execute and print the new results
update_results = connection.execute(select_stmt).fetchall()
print(update_results)

[(1, 'Laptop', 1480.3515), (2, 'Smartphone', 1174.6575), (3, 'Tablet', 506.16), (4, 'headphones', 187.59)]


# Deleting data from a database

Delete the user with `user_id = 114` from the user_table

In [37]:
# Delete  statement 
delete_stmt = delete(user_table).where(user_table.columns.user_id == 114)

# Execute the delete statement 
results = connection.execute(delete_stmt)

# chekc how many rows were affected (should be 1 if the user was deleted)
results.rowcount

0

delete all orders from the order_table that were placed before `September 5, 2023`

In [39]:
# Delete statement
delete_stmt = delete (order_table).where (order_table.columns.order_date > date(2023, 9, 5))

# Execute the delete statement 
results  = connection.execute(delete_stmt)

# chekc how many rows were affected
results.rowcount

2

delete all orders from the order_table where the quantity is greater than `500`.

In [41]:
# Delete statement 
delete_stmt = delete(order_table).where(order_table.columns.quantity > 500)

# Execute the delete statement 
results = connection.execute(delete_stmt)

# chekc how many rows were affected
results.rowcount


2

delete `all records` from both the user_table and order_table to reset those tables?

In [44]:
# Delete all records from user_table
delete_user_stmt = delete(user_table)
connection.execute(delete_user_stmt)

# Delete all records from order_table
delete_order_stmt = delete(order_table)
connection.execute(delete_order_stmt)